# Importation des Packages

In [ ]:
### Manipulation des données ###
import pandas as pd
import numpy as np

### Visualisation des données ###
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  # msno.matrix(df) pour voir les valeurs manquantes
from ydata_profiling import ProfileReport  # Génération automatique de rapports EDA

### Machine Learning - Prétraitement & Modélisation ###
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE # Gestion du déséquilibre des classes

### Machine Learning - Modèles & Sélection de Features ###
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import (
    StackingClassifier
)

from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    classification_report
)

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# EDA / Cleaning

## Importation du fichier

In [ ]:
# Importation du fichier
health = pd.read_csv(r"C:\Users\thiba\Documents\analyse heart attack japan\japan_heart_attack_dataset.csv", sep=',')

In [ ]:
# Affichage de la base
health

In [ ]:
# EDA via un ProfileReport
health_profile = ProfileReport(health, explorative=True)
health_profile

## Data Cleaning

In [ ]:
# Suppression des colonnes non utile
health.drop(columns=[col for col in health.columns if col.startswith('Extra_Column')], inplace=True)
health

# Modèle de Machine Learning

## Affichage de la base

In [ ]:
# Affichage de la base
health

In [ ]:
# Affichage des types de données
health.info()

## changement des données texte en données numérique

In [ ]:
# Remplacement des Yes/No par un booléen
bool_columns = [
    'Smoking_History', 
    'Diabetes_History', 
    'Hypertension_History', 
    'Family_History', 
    'Heart_Attack_Occurrence'
]

health[bool_columns] = health[bool_columns].replace({'Yes': True, 'No': False}).astype(bool)

In [ ]:
# Liste des colonnes catégorielles à transformer en valeurs booléennes
categorical_cols = ['Gender', 'Region', 'Physical_Activity', 'Diet_Quality', 'Alcohol_Consumption']

# Remplacement des NaN pour la consommation d'alcool en personne qui n'en consomme pas
health['Alcohol_Consumption'].fillna('Ne consomme pas', inplace=True)

# Transformation de toutes les colonnes catégorielles en booléens
health = pd.get_dummies(health, columns=categorical_cols, dtype=bool)

In [ ]:
# Vérification si tout les changements on était effectué
health.info()

## Vérification des corrélations du DataFrame

In [ ]:
matplotlib.use('TkAgg') 

# Calculer la matrice de corrélation
corr_matrix = health.corr()
# Configuration de la taille 
plt.figure(figsize=(14, 12))
# Génération de la HeatMap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# Ajout du titre
plt.title('Heatmap des corrélations du DataFrame', fontsize=16)
# Affichage
plt.show()

## Lancement du Machine Learning

In [ ]:
# 📂 Charger les données
df = health.copy()

# 🎯 Définir la cible et les features
target_column = "Heart_Attack_Occurrence"  
X = df.drop(columns=[target_column])
y = df[target_column]

# 🎚️ Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🧪 Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 📊 Sur-échantillonnage (SMOTE)
smote = SMOTE(sampling_strategy=0.7, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 🚀 Modèle de Stacking avec XGBoost et CatBoost
stacking_model = StackingClassifier(
    estimators=[
        ('xgb', XGBClassifier(eval_metric="logloss", scale_pos_weight=2.5, learning_rate=0.05, max_depth=5, n_estimators=300, colsample_bytree=0.8, subsample=0.9, random_state=42)),
        ('cat', CatBoostClassifier(verbose=0, class_weights=[1, 5], depth=6, iterations=500, learning_rate=0.03, random_state=42))
    ],
    final_estimator=LogisticRegression()
)

# Entraînement du modèle
stacking_model.fit(X_train_resampled, y_train_resampled)

# 🔍 Prédictions
y_pred_stack = stacking_model.predict(X_test)
y_prob_stack = stacking_model.predict_proba(X_test)[:, 1]

# 📊 Ajustement du seuil de décision
threshold = 0.3  # seuil ajusté pour augmenter la détection des positifs
y_pred_adjusted = (y_prob_stack > threshold).astype(int)

# 📊 Évaluation
print("\n🔹 Stacking Performance (Ajusté) avec seuil modifié :")
print("Accuracy:", accuracy_score(y_test, y_pred_adjusted))
print("AUC-ROC:", roc_auc_score(y_test, y_prob_stack))
print(classification_report(y_test, y_pred_adjusted))